In [1]:
# Core libraries
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from scipy.linalg import eigh

from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [2]:
def load_big_dataset(graph_file, outcome_file,
                                 outcome_sheet='outcomeanddemographics',
                                 outcome_col='Imp20PercentBPRS',
                                 sid_col='SID'):
    # Load outcomes
    df_labels = pd.read_excel(outcome_file, sheet_name=outcome_sheet, skiprows=1)
    df_labels[sid_col] = df_labels[sid_col].astype(str)

    feature_list = []
    sid_list = []

    # Read all sheet names from the graph file
    xls = pd.ExcelFile(graph_file)
    sheet_names = xls.sheet_names

    for sheet in sheet_names:
        sid = str(sheet)
        # Skip sheets with no outcome
        if sid not in df_labels[sid_col].values:
            continue

        df_matrix = pd.read_excel(graph_file, sheet_name=sheet, index_col=0)
        matrix = df_matrix.values
        if matrix.shape[0] != matrix.shape[1]:
            raise ValueError(f"Matrix for SID {sid} is not square!")

        triu_indices = np.triu_indices_from(matrix, k=1)
        features = matrix[triu_indices]

        feature_list.append(features)
        sid_list.append(sid)

    # Combine into DataFrame
    df_features = pd.DataFrame(feature_list, index=sid_list)
    df_features['sid'] = df_features.index

    # Merge with outcomes
    df_merged = df_features.merge(df_labels[[sid_col, outcome_col]], left_on='sid', right_on=sid_col)

    # Features and outcomes
    X = df_merged.drop(columns=['sid', sid_col, outcome_col], errors='ignore').values
    y = df_merged[outcome_col].values
    X_abs = np.abs(X)


    return X,X_abs, y, df_merged


In [ ]:
# Loading big dataset
graph_file = "/content/drive/Shared drives/GNN/SchaeferAtlas_Rest_Results_Freq_008to09_wholebrain.xlsx"
outcome_file = "/content/drive/Shared drives/GNN/RestingStateDataforAlex_3Networks.xlsx"

# Call the function
large_X, large_X_abs, large_y, large_df_merged = load_big_dataset(
    graph_file=graph_file,
    outcome_file=outcome_file,
    outcome_sheet='outcomeanddemographics',   # sheet with outcome
    outcome_col='Imp20PercentBPRS',           # outcome column
    sid_col='SID'                             # column with SIDs
)

# Check shapes
print("Features shape:", large_X.shape)
print("Outcome shape:", large_y.shape)

# Preview merged DataFrame
print(large_df_merged.head())
print(large_X)
print("x abs starts here =====")
print(large_X_abs)


KeyboardInterrupt: 

In [ ]:
import numpy as np

# Save to a single compressed file
np.savez_compressed("/content/drive/Shared drives/GNN/upper_triangle_features.npz",
                    X=large_X, X_abs=large_X_abs, y=large_y)

print("Saved X, X_abs, y to drive!")



NameError: name 'large_X' is not defined

In [3]:
data = np.load("/content/drive/Shared drives/GNN/upper_triangle_features.npz")
large_X = data['X']
large_X_abs = data['X_abs']
large_y = data['y']

print("Shapes:", large_X.shape, large_X_abs.shape, large_y.shape)


Shapes: (56, 83845) (56, 83845) (56,)


In [4]:
import pandas as pd
import numpy as np

def load_flattened_features_with_outcome(outcome_file,
                                         feature_sheet='flattenedfeatures',
                                         outcome_sheet_index=1,   # second sheet
                                         outcome_col='Imp20PercentBPRS',
                                         sid_col='SID'):

    # Load flattened features
    df_features = pd.read_excel(outcome_file, sheet_name=feature_sheet)
    df_features[sid_col] = df_features[sid_col].astype(str)

    # Load outcomes from the second sheet
    df_outcomes = pd.read_excel(outcome_file, sheet_name=outcome_sheet_index, skiprows=1)

    df_outcomes[sid_col] = df_outcomes[sid_col].astype(str)

    # Filter features to only SIDs with outcomes
    df_features_filtered = df_features[df_features[sid_col].isin(df_outcomes[sid_col].values)].copy()

    # Merge features with outcomes
    df_merged = df_features_filtered.merge(df_outcomes[[sid_col, outcome_col]], on=sid_col)

    # Split features into ABS and non-ABS
    abs_cols = [c for c in df_merged.columns if c.startswith('ABS')]
    non_abs_cols = [c for c in df_merged.columns if c not in abs_cols + [sid_col, outcome_col]]

    X_ABS = df_merged[abs_cols].values
    X = df_merged[non_abs_cols].values
    y = df_merged[outcome_col].values

    return X, X_ABS, y, df_merged


In [5]:
outcome_file = "/content/drive/Shared drives/GNN/RestingStateDataforAlex_3Networks.xlsx"

small_X, small_X_ABS, small_y, small_df_merged = load_flattened_features_with_outcome(outcome_file)

print("Non-ABS features shape:", small_X.shape)
print("ABS features shape:", small_X_ABS.shape)
print("Outcome shape:", small_y.shape)
print(small_df_merged.head())

Non-ABS features shape: (56, 105)
ABS features shape: (56, 105)
Outcome shape: (56,)
      SID  ABS_Attention_LDLPFC-Attention_RDLPFC  \
0  epp270                               0.588318   
1  epp271                               0.681967   
2  epp278                               0.357294   
3  epp309                               0.648362   
4  epp314                               0.041228   

   ABS_Attention_LDLPFC-Attention_LIPL  ABS_Attention_RDLPFC-Attention_LIPL  \
0                             0.173388                             0.072191   
1                             0.933840                             0.403052   
2                             0.175111                             0.024300   
3                             0.628166                             0.799465   
4                             0.324857                             0.329995   

   ABS_Attention_LDLPFC-Attention_RIPL  ABS_Attention_RDLPFC-Attention_RIPL  \
0                             0.233879          

In [7]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout



In [8]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

def preprocess_X(X, n_components=50):
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    pca = PCA(n_components=n_components)
    X_pca = pca.fit_transform(X_scaled)
    return X_pca

# Example for small_X
input_dim = 50  # number of PCA components
X_small_pca = preprocess_X(small_X, n_components=input_dim)
y_small = small_y


# --- MixUp function ---
def mixup(X, y, alpha=0.2):
    n_samples = X.shape[0]
    lam = np.random.beta(alpha, alpha, size=n_samples)
    indices = np.random.permutation(n_samples)
    X_mix = lam[:, None] * X + (1 - lam)[:, None] * X[indices]
    y_mix = lam * y + (1 - lam) * y[indices]
    return X_mix, y_mix



# --- Model builder ---
def build_model(n_layers, units_list, dropout_list, lr, input_dim):
    model = Sequential()
    model.add(Input(shape=(input_dim,)))
    model.add(Dense(units_list[0], activation='relu'))
    if dropout_list[0] > 0:
        model.add(Dropout(dropout_list[0]))
    for i in range(1, n_layers):
        model.add(Dense(units_list[i], activation='relu'))
        if dropout_list[i] > 0:
            model.add(Dropout(dropout_list[i]))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(
        optimizer=Adam(learning_rate=lr),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model

# --- Function to create a unique key ---
def combo_key(n_layers, units_comb, dropout_comb, lr, alpha):
    units_str = '-'.join(map(str, units_comb))
    dropout_str = '-'.join(map(str, dropout_comb))
    return f"{n_layers}|{units_str}|{dropout_str}|{lr}|{alpha}"


In [ ]:
#HP Tuning with small X dataset

import csv
import os
from itertools import product
import numpy as np
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam

# --- CSV logging setup ---
log_file = "/content/drive/Shared drives/GNN/Results/smalldataset_X_hyperparam_search_log.csv"
os.makedirs(os.path.dirname(log_file), exist_ok=True)

if os.path.exists(log_file):
    with open(log_file, "r") as f:
        reader = csv.DictReader(f)
        done_combos = {row['combo_key'] for row in reader}
else:
    done_combos = set()
    with open(log_file, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow([
            'avg_accuracy', 'mixup_avg_accuracy', 'n_layers',
            'units', 'dropout', 'learning_rate', 'mixup_alpha', 'combo_key'
        ])



# --- Search space ---
layer_options = [1, 2]
units_options = [32, 64, 128]
dropout_options = [0.0, 0.2, 0.4]
learning_rates = [1e-4, 1e-3]
mixup_alphas = [0.1, 0.2, 0.4, 0.8]

X = X_small_pca
y = y_small.values if hasattr(y_small, "values") else np.array(y_small)
input_dim = X.shape[1]

kf = KFold(n_splits=5, shuffle=True, random_state=42)

best_baseline_acc = 0
best_mixup_acc = 0
best_params = None

for n_layers in layer_options:
    for units_comb in product(units_options, repeat=n_layers):
        for dropout_comb in product(dropout_options, repeat=n_layers):
            for lr in learning_rates:
                for alpha in mixup_alphas:
                    key = combo_key(n_layers, units_comb, dropout_comb, lr, alpha)

                    if key in done_combos:
                        print(f"Skipping combo: {key}")
                        continue

                    val_accuracies = []
                    mixup_val_accuracies = []

                    for train_idx, val_idx in kf.split(X):
                        X_train, X_val = X[train_idx], X[val_idx]
                        y_train, y_val = y[train_idx], y[val_idx]

                        # --- Baseline ---
                        model_base = build_model(n_layers, units_comb, dropout_comb, lr, input_dim)
                        model_base.fit(X_train, y_train, epochs=30, batch_size=32, verbose=0)
                        _, val_acc = model_base.evaluate(X_val, y_val, verbose=0)
                        val_accuracies.append(val_acc)

                        # --- MixUp ---
                        X_mix, y_mix = mixup(X_train, y_train, alpha=alpha)
                        model_mix = build_model(n_layers, units_comb, dropout_comb, lr, input_dim)
                        model_mix.fit(X_mix, y_mix, epochs=30, batch_size=32, verbose=0)
                        _, mix_val_acc = model_mix.evaluate(X_val, y_val, verbose=0)
                        mixup_val_accuracies.append(mix_val_acc)

                    mean_acc = np.mean(val_accuracies)
                    mean_mixup_acc = np.mean(mixup_val_accuracies)

                    if mean_acc > best_baseline_acc:
                        best_baseline_acc = mean_acc
                    if mean_mixup_acc > best_mixup_acc:
                        best_mixup_acc = mean_mixup_acc
                        best_params = {
                            'n_layers': n_layers,
                            'units': units_comb,
                            'dropout': dropout_comb,
                            'learning_rate': lr,
                            'mixup_alpha': alpha
                        }

                    # --- Log to CSV ---
                    with open(log_file, "a", newline="") as f:
                        writer = csv.writer(f)
                        writer.writerow([
                            mean_acc, mean_mixup_acc, n_layers,
                            '-'.join(map(str, units_comb)), '-'.join(map(str, dropout_comb)),
                            lr, alpha, key
                        ])

                    print(f"Done combo: {key}, baseline acc: {mean_acc:.4f}, mixup acc: {mean_mixup_acc:.4f}")

print("Best baseline CV accuracy:", best_baseline_acc)
print("Best MixUp CV accuracy:", best_mixup_acc)
print("Best hyperparameters for MixUp:", best_params)


Skipping combo: 1|32|0.0|0.0001|0.1
Skipping combo: 1|32|0.0|0.0001|0.2
Skipping combo: 1|32|0.0|0.0001|0.4
Skipping combo: 1|32|0.0|0.0001|0.8
Skipping combo: 1|32|0.0|0.001|0.1
Skipping combo: 1|32|0.0|0.001|0.2
Skipping combo: 1|32|0.0|0.001|0.4
Skipping combo: 1|32|0.0|0.001|0.8
Skipping combo: 1|32|0.2|0.0001|0.1
Skipping combo: 1|32|0.2|0.0001|0.2
Skipping combo: 1|32|0.2|0.0001|0.4
Skipping combo: 1|32|0.2|0.0001|0.8
Skipping combo: 1|32|0.2|0.001|0.1
Skipping combo: 1|32|0.2|0.001|0.2
Skipping combo: 1|32|0.2|0.001|0.4
Skipping combo: 1|32|0.2|0.001|0.8
Skipping combo: 1|32|0.4|0.0001|0.1
Skipping combo: 1|32|0.4|0.0001|0.2
Skipping combo: 1|32|0.4|0.0001|0.4
Skipping combo: 1|32|0.4|0.0001|0.8
Skipping combo: 1|32|0.4|0.001|0.1
Skipping combo: 1|32|0.4|0.001|0.2
Skipping combo: 1|32|0.4|0.001|0.4
Skipping combo: 1|32|0.4|0.001|0.8
Skipping combo: 1|64|0.0|0.0001|0.1
Skipping combo: 1|64|0.0|0.0001|0.2
Skipping combo: 1|64|0.0|0.0001|0.4
Skipping combo: 1|64|0.0|0.0001|0.8
Skip

KeyboardInterrupt: 

In [ ]:
#HP Tuning with small X ABS dataset

import csv
import os
from itertools import product
import numpy as np
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam


input_dim = 50  # number of PCA components
X_small_pca = preprocess_X(small_X_ABS, n_components=input_dim)
y_small = small_y

#HP Tuning with small X dataset

import csv
import os
from itertools import product
import numpy as np
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam

# --- CSV logging setup ---
log_file = "/content/drive/Shared drives/GNN/Results/smalldataset_X_ABS_hyperparam_search_log.csv"
os.makedirs(os.path.dirname(log_file), exist_ok=True)

if os.path.exists(log_file):
    with open(log_file, "r") as f:
        reader = csv.DictReader(f)
        done_combos = {row['combo_key'] for row in reader}
else:
    done_combos = set()
    with open(log_file, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow([
            'avg_accuracy', 'mixup_avg_accuracy', 'n_layers',
            'units', 'dropout', 'learning_rate', 'mixup_alpha', 'combo_key'
        ])


# --- Search space ---
layer_options = [1, 2]
units_options = [32, 64, 128]
dropout_options = [0.0, 0.2, 0.4]
learning_rates = [1e-4, 1e-3]
mixup_alphas = [0.1, 0.2, 0.4, 0.8]

X = X_small_pca
y = y_small.values if hasattr(y_small, "values") else np.array(y_small)
input_dim = X.shape[1]

kf = KFold(n_splits=5, shuffle=True, random_state=42)

best_baseline_acc = 0
best_mixup_acc = 0
best_params = None

for n_layers in layer_options:
    for units_comb in product(units_options, repeat=n_layers):
        for dropout_comb in product(dropout_options, repeat=n_layers):
            for lr in learning_rates:
                for alpha in mixup_alphas:
                    key = combo_key(n_layers, units_comb, dropout_comb, lr, alpha)

                    if key in done_combos:
                        print(f"Skipping combo: {key}")
                        continue

                    val_accuracies = []
                    mixup_val_accuracies = []

                    for train_idx, val_idx in kf.split(X):
                        X_train, X_val = X[train_idx], X[val_idx]
                        y_train, y_val = y[train_idx], y[val_idx]

                        # --- Baseline ---
                        model_base = build_model(n_layers, units_comb, dropout_comb, lr, input_dim)
                        model_base.fit(X_train, y_train, epochs=30, batch_size=32, verbose=0)
                        _, val_acc = model_base.evaluate(X_val, y_val, verbose=0)
                        val_accuracies.append(val_acc)

                        # --- MixUp ---
                        X_mix, y_mix = mixup(X_train, y_train, alpha=alpha)
                        model_mix = build_model(n_layers, units_comb, dropout_comb, lr, input_dim)
                        model_mix.fit(X_mix, y_mix, epochs=30, batch_size=32, verbose=0)
                        _, mix_val_acc = model_mix.evaluate(X_val, y_val, verbose=0)
                        mixup_val_accuracies.append(mix_val_acc)

                    mean_acc = np.mean(val_accuracies)
                    mean_mixup_acc = np.mean(mixup_val_accuracies)

                    if mean_acc > best_baseline_acc:
                        best_baseline_acc = mean_acc
                    if mean_mixup_acc > best_mixup_acc:
                        best_mixup_acc = mean_mixup_acc
                        best_params = {
                            'n_layers': n_layers,
                            'units': units_comb,
                            'dropout': dropout_comb,
                            'learning_rate': lr,
                            'mixup_alpha': alpha
                        }

                    # --- Log to CSV ---
                    with open(log_file, "a", newline="") as f:
                        writer = csv.writer(f)
                        writer.writerow([
                            mean_acc, mean_mixup_acc, n_layers,
                            '-'.join(map(str, units_comb)), '-'.join(map(str, dropout_comb)),
                            lr, alpha, key
                        ])

                    print(f"Done combo: {key}, baseline acc: {mean_acc:.4f}, mixup acc: {mean_mixup_acc:.4f}")

print("Best baseline CV accuracy:", best_baseline_acc)
print("Best MixUp CV accuracy:", best_mixup_acc)
print("Best hyperparameters for MixUp:", best_params)


Skipping combo: 1|32|0.0|0.0001|0.1
Skipping combo: 1|32|0.0|0.0001|0.2
Skipping combo: 1|32|0.0|0.0001|0.4
Skipping combo: 1|32|0.0|0.0001|0.8
Skipping combo: 1|32|0.0|0.001|0.1
Skipping combo: 1|32|0.0|0.001|0.2
Skipping combo: 1|32|0.0|0.001|0.4
Skipping combo: 1|32|0.0|0.001|0.8
Skipping combo: 1|32|0.2|0.0001|0.1
Skipping combo: 1|32|0.2|0.0001|0.2
Skipping combo: 1|32|0.2|0.0001|0.4
Skipping combo: 1|32|0.2|0.0001|0.8
Done combo: 1|32|0.2|0.001|0.1, baseline acc: 0.4470, mixup acc: 0.3758
Done combo: 1|32|0.2|0.001|0.2, baseline acc: 0.4985, mixup acc: 0.5530
Done combo: 1|32|0.2|0.001|0.4, baseline acc: 0.4106, mixup acc: 0.5727
Done combo: 1|32|0.2|0.001|0.8, baseline acc: 0.4273, mixup acc: 0.5894
Done combo: 1|32|0.4|0.0001|0.1, baseline acc: 0.5379, mixup acc: 0.5348
Done combo: 1|32|0.4|0.0001|0.2, baseline acc: 0.4636, mixup acc: 0.3924
Done combo: 1|32|0.4|0.0001|0.4, baseline acc: 0.5182, mixup acc: 0.5030
Done combo: 1|32|0.4|0.0001|0.8, baseline acc: 0.5152, mixup acc:

In [ ]:

#HP Tuning with large X dataset

import csv
import os
from itertools import product
import numpy as np
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam


input_dim = 50  # number of PCA components
X_large_pca = preprocess_X(large_X, n_components=input_dim)
y_large = large_y

# --- CSV logging setup ---
log_file = "/content/drive/Shared drives/GNN/Results/largedataset_X_hyperparam_search_log.csv"
os.makedirs(os.path.dirname(log_file), exist_ok=True)

if os.path.exists(log_file):
    with open(log_file, "r") as f:
        reader = csv.DictReader(f)
        done_combos = {row['combo_key'] for row in reader}
else:
    done_combos = set()
    with open(log_file, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow([
            'avg_accuracy', 'mixup_avg_accuracy', 'n_layers',
            'units', 'dropout', 'learning_rate', 'mixup_alpha', 'combo_key'
        ])


# --- Search space ---
layer_options = [1, 2]
units_options = [32, 64, 128]
dropout_options = [0.0, 0.2, 0.4]
learning_rates = [1e-4, 1e-3]
mixup_alphas = [0.1, 0.2, 0.4, 0.8]

X = X_large_pca
y = y_large.values if hasattr(y_small, "values") else np.array(y_small)
input_dim = X.shape[1]

kf = KFold(n_splits=5, shuffle=True, random_state=42)

best_baseline_acc = 0
best_mixup_acc = 0
best_params = None


for n_layers in layer_options:
    for units_comb in product(units_options, repeat=n_layers):
        for dropout_comb in product(dropout_options, repeat=n_layers):
            for lr in learning_rates:
                for alpha in mixup_alphas:
                    key = combo_key(n_layers, units_comb, dropout_comb, lr, alpha)

                    if key in done_combos:
                        print(f"Skipping combo: {key}")
                        continue

                    val_accuracies = []
                    mixup_val_accuracies = []

                    for train_idx, val_idx in kf.split(X):
                        X_train, X_val = X[train_idx], X[val_idx]
                        y_train, y_val = y[train_idx], y[val_idx]

                        # --- Baseline ---
                        model_base = build_model(n_layers, units_comb, dropout_comb, lr, input_dim)
                        model_base.fit(X_train, y_train, epochs=30, batch_size=32, verbose=0)
                        _, val_acc = model_base.evaluate(X_val, y_val, verbose=0)
                        val_accuracies.append(val_acc)

                        # --- MixUp ---
                        X_mix, y_mix = mixup(X_train, y_train, alpha=alpha)
                        model_mix = build_model(n_layers, units_comb, dropout_comb, lr, input_dim)
                        model_mix.fit(X_mix, y_mix, epochs=30, batch_size=32, verbose=0)
                        _, mix_val_acc = model_mix.evaluate(X_val, y_val, verbose=0)
                        mixup_val_accuracies.append(mix_val_acc)

                    mean_acc = np.mean(val_accuracies)
                    mean_mixup_acc = np.mean(mixup_val_accuracies)

                    if mean_acc > best_baseline_acc:
                        best_baseline_acc = mean_acc
                    if mean_mixup_acc > best_mixup_acc:
                        best_mixup_acc = mean_mixup_acc
                        best_params = {
                            'n_layers': n_layers,
                            'units': units_comb,
                            'dropout': dropout_comb,
                            'learning_rate': lr,
                            'mixup_alpha': alpha
                        }

                    # --- Log to CSV ---
                    with open(log_file, "a", newline="") as f:
                        writer = csv.writer(f)
                        writer.writerow([
                            mean_acc, mean_mixup_acc, n_layers,
                            '-'.join(map(str, units_comb)), '-'.join(map(str, dropout_comb)),
                            lr, alpha, key
                        ])

                    print(f"Done combo: {key}, baseline acc: {mean_acc:.4f}, mixup acc: {mean_mixup_acc:.4f}")

print("Best baseline CV accuracy:", best_baseline_acc)
print("Best MixUp CV accuracy:", best_mixup_acc)
print("Best hyperparameters for MixUp:", best_params)


Skipping combo: 1|32|0.0|0.0001|0.1
Skipping combo: 1|32|0.0|0.0001|0.2
Skipping combo: 1|32|0.0|0.0001|0.4
Skipping combo: 1|32|0.0|0.0001|0.8
Skipping combo: 1|32|0.0|0.001|0.1
Done combo: 1|32|0.0|0.001|0.2, baseline acc: 0.5015, mixup acc: 0.5212
Done combo: 1|32|0.0|0.001|0.4, baseline acc: 0.5197, mixup acc: 0.4121
Done combo: 1|32|0.0|0.001|0.8, baseline acc: 0.5530, mixup acc: 0.4985
Done combo: 1|32|0.2|0.0001|0.1, baseline acc: 0.5545, mixup acc: 0.5197
Done combo: 1|32|0.2|0.0001|0.2, baseline acc: 0.6106, mixup acc: 0.4667
Done combo: 1|32|0.2|0.0001|0.4, baseline acc: 0.3939, mixup acc: 0.5030
Done combo: 1|32|0.2|0.0001|0.8, baseline acc: 0.5879, mixup acc: 0.4833
Done combo: 1|32|0.2|0.001|0.1, baseline acc: 0.6939, mixup acc: 0.5167
Done combo: 1|32|0.2|0.001|0.2, baseline acc: 0.5167, mixup acc: 0.5015
Done combo: 1|32|0.2|0.001|0.4, baseline acc: 0.5182, mixup acc: 0.4955
Done combo: 1|32|0.2|0.001|0.8, baseline acc: 0.6061, mixup acc: 0.5576
Done combo: 1|32|0.4|0.00

KeyboardInterrupt: 

In [ ]:

#HP Tuning with large X ABS dataset

import csv
import os
from itertools import product
import numpy as np
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam


input_dim = 50  # number of PCA components
X_large_pca = preprocess_X(large_X_abs, n_components=input_dim)
y_large = large_y

# --- CSV logging setup ---
log_file = "/content/drive/Shared drives/GNN/Results/largedataset_X_ABS_hyperparam_search_log.csv"
os.makedirs(os.path.dirname(log_file), exist_ok=True)

if os.path.exists(log_file):
    with open(log_file, "r") as f:
        reader = csv.DictReader(f)
        done_combos = {row['combo_key'] for row in reader}
else:
    done_combos = set()
    with open(log_file, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow([
            'avg_accuracy', 'mixup_avg_accuracy', 'n_layers',
            'units', 'dropout', 'learning_rate', 'mixup_alpha', 'combo_key'
        ])


# --- Search space ---
layer_options = [1, 2]
units_options = [32, 64, 128]
dropout_options = [0.0, 0.2, 0.4]
learning_rates = [1e-4, 1e-3]
mixup_alphas = [0.1, 0.2, 0.4, 0.8]

X = X_large_pca
y = y_large.values if hasattr(y_small, "values") else np.array(y_small)
input_dim = X.shape[1]

kf = KFold(n_splits=5, shuffle=True, random_state=42)

best_baseline_acc = 0
best_mixup_acc = 0
best_params = None


for n_layers in layer_options:
    for units_comb in product(units_options, repeat=n_layers):
        for dropout_comb in product(dropout_options, repeat=n_layers):
            for lr in learning_rates:
                for alpha in mixup_alphas:
                    key = combo_key(n_layers, units_comb, dropout_comb, lr, alpha)

                    if key in done_combos:
                        print(f"Skipping combo: {key}")
                        continue

                    val_accuracies = []
                    mixup_val_accuracies = []

                    for train_idx, val_idx in kf.split(X):
                        X_train, X_val = X[train_idx], X[val_idx]
                        y_train, y_val = y[train_idx], y[val_idx]

                        # --- Baseline ---
                        model_base = build_model(n_layers, units_comb, dropout_comb, lr, input_dim)
                        model_base.fit(X_train, y_train, epochs=30, batch_size=32, verbose=0)
                        _, val_acc = model_base.evaluate(X_val, y_val, verbose=0)
                        val_accuracies.append(val_acc)

                        # --- MixUp ---
                        X_mix, y_mix = mixup(X_train, y_train, alpha=alpha)
                        model_mix = build_model(n_layers, units_comb, dropout_comb, lr, input_dim)
                        model_mix.fit(X_mix, y_mix, epochs=30, batch_size=32, verbose=0)
                        _, mix_val_acc = model_mix.evaluate(X_val, y_val, verbose=0)
                        mixup_val_accuracies.append(mix_val_acc)

                    mean_acc = np.mean(val_accuracies)
                    mean_mixup_acc = np.mean(mixup_val_accuracies)

                    if mean_acc > best_baseline_acc:
                        best_baseline_acc = mean_acc
                    if mean_mixup_acc > best_mixup_acc:
                        best_mixup_acc = mean_mixup_acc
                        best_params = {
                            'n_layers': n_layers,
                            'units': units_comb,
                            'dropout': dropout_comb,
                            'learning_rate': lr,
                            'mixup_alpha': alpha
                        }

                    # --- Log to CSV ---
                    with open(log_file, "a", newline="") as f:
                        writer = csv.writer(f)
                        writer.writerow([
                            mean_acc, mean_mixup_acc, n_layers,
                            '-'.join(map(str, units_comb)), '-'.join(map(str, dropout_comb)),
                            lr, alpha, key
                        ])

                    print(f"Done combo: {key}, baseline acc: {mean_acc:.4f}, mixup acc: {mean_mixup_acc:.4f}")

print("Best baseline CV accuracy:", best_baseline_acc)
print("Best MixUp CV accuracy:", best_mixup_acc)
print("Best hyperparameters for MixUp:", best_params)


Done combo: 1|32|0.0|0.0001|0.1, baseline acc: 0.4864, mixup acc: 0.4803
Done combo: 1|32|0.0|0.0001|0.2, baseline acc: 0.5515, mixup acc: 0.5379
Done combo: 1|32|0.0|0.0001|0.4, baseline acc: 0.5348, mixup acc: 0.5545
Done combo: 1|32|0.0|0.0001|0.8, baseline acc: 0.4818, mixup acc: 0.4303
Done combo: 1|32|0.0|0.001|0.1, baseline acc: 0.5227, mixup acc: 0.5758
Done combo: 1|32|0.0|0.001|0.2, baseline acc: 0.5712, mixup acc: 0.5561
Done combo: 1|32|0.0|0.001|0.4, baseline acc: 0.5167, mixup acc: 0.5545
Done combo: 1|32|0.0|0.001|0.8, baseline acc: 0.4970, mixup acc: 0.5182
Done combo: 1|32|0.2|0.0001|0.1, baseline acc: 0.5545, mixup acc: 0.5197
Done combo: 1|32|0.2|0.0001|0.2, baseline acc: 0.4818, mixup acc: 0.5318
Done combo: 1|32|0.2|0.0001|0.4, baseline acc: 0.3939, mixup acc: 0.5136
Done combo: 1|32|0.2|0.0001|0.8, baseline acc: 0.6394, mixup acc: 0.5894
Done combo: 1|32|0.2|0.001|0.1, baseline acc: 0.5318, mixup acc: 0.4970
Done combo: 1|32|0.2|0.001|0.2, baseline acc: 0.4985, mi

In [23]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tqdm import tqdm



n_layers = 1
units = [32]
dropout = [0.4]
learning_rate = 0.0001
mixup_alpha = 0.8

#X_pca = preprocess_X(small_X,n_components=input_dim) #small X
#X_pca = preprocess_X(small_X_ABS, n_components=input_dim) #small X abs
#X_pca = preprocess_X(large_X,n_components=input_dim) #large X
X_pca = preprocess_X(large_X_abs,n_components=input_dim) #large X abs



X = X_pca
y = y_small.values if hasattr(y_small, "values") else np.array(y_small)
input_dim = X.shape[1]

# --- Logging ---
pred_log = []
all_metrics = []

n_repeats = 10
n_splits = 5




for repeat in tqdm(range(n_repeats), desc="Repeats"):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=repeat)

    for fold_idx, (train_idx, test_idx) in enumerate(kf.split(X)):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        # --- Baseline ---
        model_baseline = build_model(n_layers, units, dropout, learning_rate, input_dim)
        model_baseline.fit(X_train, y_train, epochs=30, batch_size=32, verbose=0)
        y_pred_baseline = (model_baseline.predict(X_test) > 0.5).astype(int).flatten()

        # --- MixUp ---
        X_mix, y_mix = mixup(X_train, y_train, alpha=mixup_alpha)
        model_mix = build_model(n_layers, units, dropout, learning_rate, input_dim)
        model_mix.fit(X_mix, y_mix, epochs=30, batch_size=32, verbose=0)
        y_pred_mix = (model_mix.predict(X_test) > 0.5).astype(int).flatten()

        # --- Store individual predictions ---
        for i, idx in enumerate(test_idx):
            pred_log.append({
                "run": repeat,
                "fold": fold_idx,
                "idx": idx,
                "true_label": y_test[i],
                "pred_baseline": y_pred_baseline[i],
                "pred_mixup": y_pred_mix[i]
            })

        # --- Compute metrics ---
        metrics = {
            "run": repeat,
            "fold": fold_idx,
            "acc_baseline": accuracy_score(y_test, y_pred_baseline),
            "acc_mixup": accuracy_score(y_test, y_pred_mix),
            "acc_baseline_improvers": accuracy_score(y_test[y_test==1], y_pred_baseline[y_test==1]) if sum(y_test==1)>0 else np.nan,
            "acc_mixup_improvers": accuracy_score(y_test[y_test==1], y_pred_mix[y_test==1]) if sum(y_test==1)>0 else np.nan,
            "acc_baseline_nonimprovers": accuracy_score(y_test[y_test==0], y_pred_baseline[y_test==0]) if sum(y_test==0)>0 else np.nan,
            "acc_mixup_nonimprovers": accuracy_score(y_test[y_test==0], y_pred_mix[y_test==0]) if sum(y_test==0)>0 else np.nan,
            "precision_baseline": precision_score(y_test, y_pred_baseline, zero_division=0),
            "precision_mixup": precision_score(y_test, y_pred_mix, zero_division=0),
            "recall_baseline": recall_score(y_test, y_pred_baseline, zero_division=0),
            "recall_mixup": recall_score(y_test, y_pred_mix, zero_division=0),
            "f1_baseline": f1_score(y_test, y_pred_baseline, zero_division=0),
            "f1_mixup": f1_score(y_test, y_pred_mix, zero_division=0),
            "roc_auc_baseline": roc_auc_score(y_test, y_pred_baseline),
            "roc_auc_mixup": roc_auc_score(y_test, y_pred_mix)
        }
        all_metrics.append(metrics)

# --- Convert logs to DataFrames ---
pred_df = pd.DataFrame(pred_log)
metrics_df = pd.DataFrame(all_metrics)

# --- Save individual predictions & fold metrics ---
pred_df.to_csv("/content/drive/Shared drives/GNN/Results/predictions_log.csv", index=False)
metrics_df.to_csv("/content/drive/Shared drives/GNN/Results/fold_metrics.csv", index=False)

# --- Compute summary with 95% CI ---
def compute_ci(series, alpha=0.05):
    lower = np.percentile(series, 100*alpha/2)
    upper = np.percentile(series, 100*(1-alpha/2))
    mean = np.mean(series)
    return mean, lower, upper

summary_metrics = {}
for col in metrics_df.columns:
    if col not in ['run', 'fold']:
        mean, lower, upper = compute_ci(metrics_df[col].dropna())
        summary_metrics[col] = {"mean": mean, "95%_CI_lower": lower, "95%_CI_upper": upper}

summary_df = pd.DataFrame(summary_metrics).T
summary_df.to_csv("/content/drive/Shared drives/GNN/Results/summary_metrics_with_CI.csv")
print(summary_df)









Repeats:   0%|          | 0/10 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step


Repeats:  10%|█         | 1/10 [00:29<04:29, 29.94s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step


Repeats:  20%|██        | 2/10 [01:03<04:16, 32.01s/it]

...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step


Repeats:  90%|█████████ | 9/10 [04:30<00:29, 29.20s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step


Repeats: 100%|██████████| 10/10 [05:03<00:00, 30.37s/it]

                               mean  95%_CI_lower  95%_CI_upper
acc_baseline               0.503333      0.181818      0.818182
acc_mixup                  0.492576      0.272727      0.809470
acc_baseline_improvers     0.537317      0.028125      1.000000
acc_mixup_improvers        0.488921      0.000000      1.000000
acc_baseline_nonimprovers  0.466421      0.000000      1.000000
acc_mixup_nonimprovers     0.509175      0.000000      1.000000
precision_baseline         0.516114      0.028125      1.000000
precision_mixup            0.505055      0.000000      1.000000
recall_baseline            0.537317      0.028125      1.000000
recall_mixup               0.488921      0.000000      1.000000
f1_baseline                0.486194      0.040909      0.849580
f1_mixup                   0.448922      0.000000      0.840132
roc_auc_baseline           0.501869      0.189509      0.791518
roc_auc_mixup              0.499048      0.283869      0.814583


In [18]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score




# --- Data ---
X_pca = preprocess_X(small_X,n_components=input_dim) #small X
#X_pca = preprocess_X(small_X_ABS, n_components=input_dim) #small X abs
#X_pca = preprocess_X(large_X,n_components=input_dim) #large X
#X_pca = preprocess_X(large_X_abs,n_components=input_dim) #large X abs

y = y_small.values if hasattr(y_small, "values") else np.array(y_small)

# --- Repeated CV setup ---
n_splits = 5
n_repeats = 30
mixup_alpha = 0.4

# Storage
all_fold_results = []
all_predictions = []

for repeat in range(1, n_repeats+1):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=repeat)

    for fold, (train_idx, test_idx) in enumerate(kf.split(X_pca), 1):
        X_train, X_test = X_pca[train_idx], X_pca[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        # --- Baseline SVR ---
        svr_base = SVR(kernel='rbf', C=1.0, epsilon=0.1)
        svr_base.fit(X_train, y_train)
        y_pred_base_cont = svr_base.predict(X_test)
        y_pred_base = (y_pred_base_cont > 0.5).astype(int)  # threshold for classification

        # --- MixUp SVR ---
        X_mix, y_mix = mixup(X_train, y_train, alpha=mixup_alpha)
        svr_mix = SVR(kernel='rbf', C=1.0, epsilon=0.1)
        svr_mix.fit(X_mix, y_mix)
        y_pred_mix_cont = svr_mix.predict(X_test)
        y_pred_mix = (y_pred_mix_cont > 0.5).astype(int)

        # --- Track metrics ---
        fold_results = {
            'repeat': repeat,
            'fold': fold,
            'acc_baseline': accuracy_score(y_test, y_pred_base),
            'acc_mixup': accuracy_score(y_test, y_pred_mix),
            'precision_baseline': precision_score(y_test, y_pred_base, zero_division=0),
            'precision_mixup': precision_score(y_test, y_pred_mix, zero_division=0),
            'recall_baseline': recall_score(y_test, y_pred_base, zero_division=0),
            'recall_mixup': recall_score(y_test, y_pred_mix, zero_division=0),
            'f1_baseline': f1_score(y_test, y_pred_base, zero_division=0),
            'f1_mixup': f1_score(y_test, y_pred_mix, zero_division=0),
        }

        # ROC AUC needs probability, for SVR we can use raw predictions as proxy
        try:
            fold_results['roc_auc_baseline'] = roc_auc_score(y_test, y_pred_base_cont)
            fold_results['roc_auc_mixup'] = roc_auc_score(y_test, y_pred_mix_cont)
        except ValueError:
            fold_results['roc_auc_baseline'] = np.nan
            fold_results['roc_auc_mixup'] = np.nan

        all_fold_results.append(fold_results)

        # --- Save individual predictions ---
        for i, idx in enumerate(test_idx):
            all_predictions.append({
                'repeat': repeat,
                'fold': fold,
                'index': idx,
                'y_true': y_test[i],
                'y_pred_baseline': y_pred_base[i],
                'y_pred_mixup': y_pred_mix[i],
                'y_pred_baseline_cont': y_pred_base_cont[i],
                'y_pred_mixup_cont': y_pred_mix_cont[i]
            })

# --- Convert to DataFrames ---
df_results = pd.DataFrame(all_fold_results)
pred_df = pd.DataFrame(all_predictions)

# --- Compute summary metrics and 95% CI ---
summary = df_results.agg(['mean', 'std'])
n = n_splits * n_repeats
ci_95 = 1.96 * summary.loc['std'] / np.sqrt(n)
summary.loc['95_CI_lower'] = summary.loc['mean'] - ci_95
summary.loc['95_CI_upper'] = summary.loc['mean'] + ci_95

# --- Save predictions ---
pred_df.to_csv("/content/drive/Shared drives/GNN/Results/svr_repeated_fold_predictions.csv", index=False)

print("Repeated CV metrics summary (mean ± 95% CI):")
print(summary)


Repeated CV metrics summary (mean ± 95% CI):
                repeat      fold  acc_baseline  acc_mixup  precision_baseline  \
mean         15.500000  3.000000      0.557727   0.539343            0.603452   
std           8.684438  1.418951      0.146817   0.141835            0.232565   
95_CI_lower  14.110200  2.772920      0.534232   0.516645            0.566234   
95_CI_upper  16.889800  3.227080      0.581223   0.562042            0.640670   

             precision_mixup  recall_baseline  recall_mixup  f1_baseline  \
mean                0.593107         0.604988      0.679038     0.563769   
std                 0.219272         0.235582      0.270090     0.173696   
95_CI_lower         0.558016         0.567287      0.635814     0.535972   
95_CI_upper         0.628198         0.642689      0.722261     0.591567   

             f1_mixup  roc_auc_baseline  roc_auc_mixup  
mean         0.574047          0.629748       0.613140  
std          0.160109          0.163484       0.164829

In [22]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# --- Data ---

y = y_small.values if hasattr(y_small, "values") else np.array(y_small)

X_pca = preprocess_X(small_X,n_components=input_dim) #small X
#X_pca = preprocess_X(small_X_ABS, n_components=input_dim) #small X abs
#X_pca = preprocess_X(large_X,n_components=input_dim) #large X
#X_pca = preprocess_X(large_X_abs,n_components=input_dim) #large X abs

# --- Repeated CV setup ---
n_splits = 5
n_repeats = 30
mixup_alpha = 0.4

# Storage
all_fold_results = []
all_predictions = []

for repeat in range(1, n_repeats+1):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=repeat)

    for fold, (train_idx, test_idx) in enumerate(kf.split(X_pca), 1):
        X_train, X_test = X_pca[train_idx], X_pca[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        # --- Baseline MLPRegressor ---
        mlp_base = MLPRegressor(hidden_layer_sizes=(100, 100),
                                activation='relu',
                                solver='adam',
                                max_iter=500,
                                random_state=repeat)
        mlp_base.fit(X_train, y_train)
        y_pred_base_cont = mlp_base.predict(X_test)
        y_pred_base = (y_pred_base_cont > 0.5).astype(int)  # threshold for classification

        # --- MixUp MLPRegressor ---
        X_mix, y_mix = mixup(X_train, y_train, alpha=mixup_alpha)
        mlp_mix = MLPRegressor(hidden_layer_sizes=(100, 100),
                               activation='relu',
                               solver='adam',
                               max_iter=500,
                               random_state=repeat)
        mlp_mix.fit(X_mix, y_mix)
        y_pred_mix_cont = mlp_mix.predict(X_test)
        y_pred_mix = (y_pred_mix_cont > 0.5).astype(int)

        # --- Track metrics ---
        fold_results = {
            'repeat': repeat,
            'fold': fold,
            'acc_baseline': accuracy_score(y_test, y_pred_base),
            'acc_mixup': accuracy_score(y_test, y_pred_mix),
            'precision_baseline': precision_score(y_test, y_pred_base, zero_division=0),
            'precision_mixup': precision_score(y_test, y_pred_mix, zero_division=0),
            'recall_baseline': recall_score(y_test, y_pred_base, zero_division=0),
            'recall_mixup': recall_score(y_test, y_pred_mix, zero_division=0),
            'f1_baseline': f1_score(y_test, y_pred_base, zero_division=0),
            'f1_mixup': f1_score(y_test, y_pred_mix, zero_division=0),
        }

        # ROC AUC
        try:
            fold_results['roc_auc_baseline'] = roc_auc_score(y_test, y_pred_base_cont)
            fold_results['roc_auc_mixup'] = roc_auc_score(y_test, y_pred_mix_cont)
        except ValueError:
            fold_results['roc_auc_baseline'] = np.nan
            fold_results['roc_auc_mixup'] = np.nan

        all_fold_results.append(fold_results)

        # --- Save individual predictions ---
        for i, idx in enumerate(test_idx):
            all_predictions.append({
                'repeat': repeat,
                'fold': fold,
                'index': idx,
                'y_true': y_test[i],
                'y_pred_baseline': y_pred_base[i],
                'y_pred_mixup': y_pred_mix[i],
                'y_pred_baseline_cont': y_pred_base_cont[i],
                'y_pred_mixup_cont': y_pred_mix_cont[i]
            })

# --- Convert to DataFrames ---
df_results = pd.DataFrame(all_fold_results)
pred_df = pd.DataFrame(all_predictions)

# --- Compute summary metrics and 95% CI ---
summary = df_results.agg(['mean', 'std'])
n = n_splits * n_repeats
ci_95 = 1.96 * summary.loc['std'] / np.sqrt(n)
summary.loc['95_CI_lower'] = summary.loc['mean'] - ci_95
summary.loc['95_CI_upper'] = summary.loc['mean'] + ci_95

# --- Save predictions ---
pred_df.to_csv("/content/drive/Shared drives/GNN/Results/mlp_repeated_fold_predictions.csv", index=False)

print("Repeated CV metrics summary (mean ± 95% CI):")
print(summary)


Repeated CV metrics summary (mean ± 95% CI):
                repeat      fold  acc_baseline  acc_mixup  precision_baseline  \
mean         15.500000  3.000000      0.514949   0.505556            0.537915   
std           8.684438  1.418951      0.152202   0.141783            0.238627   
95_CI_lower  14.110200  2.772920      0.490592   0.482866            0.499727   
95_CI_upper  16.889800  3.227080      0.539307   0.528246            0.576104   

             precision_mixup  recall_baseline  recall_mixup  f1_baseline  \
mean                0.518627         0.491091      0.537472     0.480252   
std                 0.213605         0.256151      0.249791     0.205497   
95_CI_lower         0.484443         0.450098      0.497497     0.447366   
95_CI_upper         0.552811         0.532083      0.577447     0.513139   

             f1_mixup  roc_auc_baseline  roc_auc_mixup  
mean         0.499452          0.537022       0.529134  
std          0.194018          0.186163       0.197107